# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902071


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:40,  1.40s/it]

Rendering models:  10%|▉         | 3/31 [00:05<00:50,  1.81s/it]

Rendering models:  13%|█▎        | 4/31 [00:05<00:34,  1.30s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:37,  1.46s/it]

...

Rendering models:  74%|███████▍  | 23/31 [00:09<00:00,  8.12it/s]

Rendering models:  77%|███████▋  | 24/31 [00:09<00:01,  6.85it/s]

Rendering models:  84%|████████▍ | 26/31 [00:10<00:00,  7.62it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  9.21it/s]

equidad1                              0.000130
not-logged-in-cae133427d99aac1e554    0.000109
kayleebug2017                         0.000513
not-logged-in-0b0736c8991a7cc2017a    0.000124
ElisabethB                            0.004388
Drew_Farnsworth                       0.000108
Thedunkmasta                          0.001937
BirdLeaf                              0.000087
not-logged-in-460b94aefeb0ead1b0f0    0.375562
HexBerry                              0.000121
not-logged-in-846e14e9f3e0f594a568    0.000264
DrJackie                              0.000097
not-logged-in-66748d0532f903be0737    0.000660
not-logged-in-66748d0532f903be0737    0.008137
jmartinez468                          0.004708
not-logged-in-300dca7f45d1a5d1dd58    0.000146
pangeli5                              0.000580
helios1235                            0.000078
Lavadude                              0.090635
ShirleyL                              0.000109
sirtylerli                            0.000085
brigi.baumann

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())